In [1]:
import h5py

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# models
MODEL_PATH = '/home/ankbzpx/Documents/'

discrete_encoder_path = MODEL_PATH + 'discrete continuous shape/discrete_encoder.pth'
discrete_decoder_path = MODEL_PATH + 'discrete continuous shape/discrete_decoder.pth'
unet_path = MODEL_PATH + 'discrete continuous shape/con_unet_full.pth'
continuous_model_path = MODEL_PATH + 'discrete continuous shape/continuous_model.pth'

hidden_dim_discrete = 128

# data preparation
data_file = '/home/ankbzpx/datasets/ShapeNet/ShapeNetRenderingh5_v1/03001627/sdf_train_core.h5'
sample_size = 2048
batch_size = 12
split_ratio = 0.9
depth_size = 256
num_of_workers = 12
# training
num_epoch = 40

In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import transforms
import torch.nn as nn
import torchvision.transforms.functional as TF
from torch.autograd import Variable
from PIL import Image
import sys
import time
import matplotlib.pyplot as plt
from torchsummary import summary

In [5]:
# reproducible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
torch.manual_seed(0)
np.random.seed(0)

In [6]:
device

device(type='cuda')

In [7]:
class ChairSDFDataset(Dataset):
     
    def __init__(self, h5_file):
        
        self.file_path = h5_file
        self.dataset = None
        
        with h5py.File(self.file_path, 'r') as file:
            self.dataset_len = len(file)
            self.keys = list(file.keys())
            
        self.to_tensor = transforms.ToTensor()
     
    def __len__(self):
        return self.dataset_len
 
    def __getitem__(self, idx):
        
        #start_time = time.time()
        
        if self.dataset is None:
            self.dataset = h5py.File(self.file_path, 'r')
         
        group = self.dataset[self.keys[idx]]
        
        depth_img = self.to_tensor(Image.fromarray(np.array(group['depth_img'])))
        
        #print("--- depth preprocessing %s seconds ---" % (time.time() - start_time))
        
        sample_pt_np = np.array(group['sample_pt']).reshape(-1, 3)
        sample_sdf_np = np.array(group['sample_sdf']).reshape(-1, 1)
        
        # check size correctness and fix incorrect data
        if sample_pt_np.shape[0] != 2048:
            sample_pt_np = np.pad(sample_pt_np, ((0, 2048 - sample_pt_np.shape[0]), (0, 0)), 'reflect')
        if sample_sdf_np.shape[0] != 2048:
            sample_sdf_np = np.pad(sample_sdf_np, ((0, 2048 - sample_sdf_np.shape[0]), (0, 0)), 'reflect')
            
        
        sample_pt = torch.from_numpy(sample_pt_np).float()
        sample_sdf = torch.from_numpy(sample_sdf_np).float()
        # scale sdf
        sample_sdf = torch.sign(sample_sdf)*torch.pow(torch.abs(sample_sdf), 0.25)
        
        #print("--- subsampling %s seconds ---" % (time.time() - start_time))
        
        target_vox = torch.from_numpy(np.array(group['target_vox'])).float()
        
        sample = { 'depth_img': depth_img,
                   'sample_pt':sample_pt,
                   'sample_sdf':sample_sdf,
                   'target_vox':target_vox,
                  }
        
        return sample

In [8]:
train_sdf_dataset = ChairSDFDataset(data_file)

train_sdf_dataloader = DataLoader(train_sdf_dataset, batch_size=batch_size, shuffle=True, num_workers=num_of_workers)

In [9]:
start_epoch = 0
latent_dim = 256

from models import Discrete_encoder, Discrete_decoder, Conditional_UNET, Continuous

####################
# Discrete Encoder #
####################

discrete_encoder = Discrete_encoder(256).to(device)
discrete_encoder.load_state_dict(torch.load(discrete_encoder_path))
discrete_encoder.eval()

for child in discrete_encoder.children():
    for param in child.parameters():
        param.requires_grad = False

        
# ####################
# # Discrete Decoder #
# ####################
        
discrete_decoder = Discrete_decoder(256).to(device)
discrete_decoder.load_state_dict(torch.load(discrete_decoder_path))
discrete_decoder.eval()

for child in discrete_decoder.children():
    for param in child.parameters():
        param.requires_grad = False
        
########
# UNET #
########

# pre-trained model is loaded within the model
unet = Conditional_UNET(unet_path).to(device)
unet.eval()

for child in unet.children():
    for param in child.parameters():
        param.requires_grad = False

In [10]:
class BottleNect1D(nn.Module):
    def __init__(self, input_dim, expand = 5):
        super(BottleNect1D, self).__init__()
        
        self.block = nn.Sequential(
            nn.Linear(input_dim, expand*input_dim),
            nn.BatchNorm1d(expand*input_dim),
            nn.ReLU(),
            nn.Linear(expand*input_dim, input_dim),
            nn.BatchNorm1d(input_dim),
        )
    
    def forward(self, x):
        return x + self.block(x)

class Continuous(nn.Module):
    def __init__(self, pt_dim = 3, con_dim = 32, latent_dim = 256):
        super(Continuous, self).__init__()
        
        self.de_pt =  nn.Sequential(
            nn.Linear(pt_dim + con_dim, 64),
            nn.BatchNorm1d(64),
            BottleNect1D(64),
            nn.Linear(64, latent_dim),
            nn.BatchNorm1d(latent_dim),
            BottleNect1D(latent_dim),
        )
        
        self.de_1 = nn.Sequential(
            nn.Linear(2*latent_dim, 2*latent_dim),
            nn.BatchNorm1d(2*latent_dim),
            BottleNect1D(2*latent_dim),
            nn.Linear(2*latent_dim, 2*latent_dim),
            nn.BatchNorm1d(2*latent_dim),
            BottleNect1D(2*latent_dim),
        )
        
        self.de_2 = nn.Sequential(
            nn.Linear(4*latent_dim, 2*latent_dim),
            nn.BatchNorm1d(2*latent_dim),
            BottleNect1D(2*latent_dim),
            nn.Linear(2*latent_dim, 2*latent_dim),
            nn.BatchNorm1d(2*latent_dim),
            BottleNect1D(2*latent_dim),
            nn.Linear(2*latent_dim, 1),
            nn.Tanh(),
        )
        
    def forward(self, pt, con, z):
        
        cat = torch.cat((self.de_pt(torch.cat((pt, con), 1)), z), 1)
        out = self.de_1(cat)
        out = self.de_2(torch.cat((out, cat), 1))
        
        return out

In [11]:
continuous = Continuous().to(device)

In [12]:
l1loss = nn.L1Loss()

In [13]:
lr = 3e-4

model = continuous

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [14]:
def save_checkpoint(state):
    torch.save(state, 'continuous_model_checkpoint.pth.tar')

In [14]:
checkpoint = torch.load('continuous_model_checkpoint.pth.tar')
start_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['continuous_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [15]:
# advanced indexing 2x2x2 context from voxel
def getContext(sample_pt_query, vox):
    
    # sample_pt bxmx3
    # vox bxcxdimxdimxdim
    
    channel_size = vox.shape[1]
    batch_size, sample_size, _ = sample_pt_query.shape
    meshgrid_base = torch.Tensor(np.meshgrid(np.arange(0, batch_size), np.arange(0, channel_size), np.arange(0, 2), np.arange(0, 2), np.arange(0, 2))).int()
    context = torch.empty((batch_size, sample_size, channel_size, 2, 2, 2))
    
    for j in range(context.shape[1]):
        context[:, j, :, :, :, :] = vox[
                    meshgrid_base[0].long(),
                    meshgrid_base[1].long(),
                    (meshgrid_base[2] + sample_pt_query[:, j, 0].reshape(1, -1, 1, 1, 1)).long(), 
                    (meshgrid_base[3] + sample_pt_query[:, j, 1].reshape(1, -1, 1, 1, 1)).long(), 
                    (meshgrid_base[4] + sample_pt_query[:, j, 2].reshape(1, -1, 1, 1, 1)).long()
                ].transpose(0, 1)
    
    # b x c x m x 2 x 2 x 2
    return context.transpose(1, 2)

In [16]:
def trilinearInterpolation(context, dx, dy, dz):
    
    v0 = context[:, :, :, 0, 0, 0]*(1-dx)*(1-dy)*(1-dz)
    v1 = context[:, :, :, 1, 0, 0]*dx*(1-dy)*(1-dz)
    v2 = context[:, :, :, 0, 1, 0]*(1-dx)*dy*(1-dz)
    v3 = context[:, :, :, 1, 1, 0]*dx*dy*(1-dz)
    v4 = context[:, :, :, 0, 0, 1]*(1-dx)*(1-dy)*dz
    v5 = context[:, :, :, 1, 0, 1]*dx*(1-dy)*dz
    v6 = context[:, :, :, 0, 1, 1]*(1-dx)*dy*dz
    v7 = context[:, :, :, 1, 1, 1]*dx*dy*dz
    
    # b x c x m 1
    return v0 + v1 + v2 + v3 + v4 + v5 + v6 + v7

In [17]:
#########################################
#test sub module

import time

vox_size = 32
latent_dim = 256
con_dim = 32

batch_len = len(train_sdf_dataloader)

print("Starting Training Loop...")

start_time = time.time()

for epoch in range(start_epoch, num_epoch):
    
    loss_list = []
    loss_batch = []
    
    print("Epoch: ", epoch)
    
    count = 0
    
    for i, data in enumerate(train_sdf_dataloader):
        
        ####################
        # Data preparation #
        ####################
        
        # b x 1 x 256 x 256
        depth_img = data['depth_img'].to(device)
        # b x 128 x 1 x 1
        z = discrete_encoder(depth_img)
        # b x n x 3
        # DO NOT scale by np.sqrt(3)
        sample_pt = data['sample_pt']
        # b x n x 1
        sample_sdf = data['sample_sdf']
        
        # b x 16 x 64 x 64 x 64
        target_vox = data['target_vox'].to(device)
        vox_feature = unet(target_vox, z)
        #vox_feature = unet(torch.sigmoid(discrete_decoder(mapping(z))))
        
        ####################
        # indexing context #
        ####################
        
        # stay with cpu for v-ram efficiency
        sample_pt_normalized = sample_pt + torch.tensor([0.5, 0.5, 0.5])
        # (0, vox_size-1)
        sample_pt_scale = torch.clamp(sample_pt_normalized* (vox_size-1), 0, (vox_size-1)-1e-5)
        # (0, vox_size-2)
        sample_pt_query = torch.clamp((sample_pt_scale).int(), 0, (vox_size-2))
        sample_pt_distance = sample_pt_scale - sample_pt_query
        
        context = getContext(sample_pt_query, vox_feature)
        
        dx = sample_pt_distance[:, :, 0].unsqueeze(1)
        dy = sample_pt_distance[:, :, 1].unsqueeze(1)
        dz = sample_pt_distance[:, :, 2].unsqueeze(1)
        # local feature
        con = trilinearInterpolation(context, dx, dy, dz)
        
        ################################
        # Reshape input & forward pass #
        ################################
        
        sample_pt = sample_pt.transpose(-1, -2).to(device)
        con = con.to(device)
        z = z.squeeze(-1).squeeze(-1).repeat(1, 1, sample_size)
        sample_sdf = sample_sdf.transpose(-1, -2).to(device)
        
        
        sample_pt = sample_pt.transpose(-1, -2).reshape(-1, 3)
        con = con.transpose(-1, -2).reshape(-1, con_dim)
        z = z.transpose(-1, -2).reshape(-1, latent_dim)
        sample_sdf = sample_sdf.transpose(-1, -2).reshape(-1, 1)
        
        
        optimizer.zero_grad()
        
        pred_sdf = model(sample_pt, con, z)
        
        loss_l1 = l1loss(pred_sdf, sample_sdf)
        
        loss = loss_l1
        
        loss.backward()
        
        loss_list.append(loss_l1.item())
        loss_batch.append(loss_l1.item())
        
        optimizer.step()
        
        #scheduler.step()
        
        if count != 0 and count % 10 == 0:
            loss_batch_avg = np.average(loss_batch)
            
            print("Batch: ", count, ", l1 Loss: ", loss_batch_avg, ", Time: %s s" % (time.time() - start_time))
            
            if count % 500 == 0:
                torch.save(model.state_dict(), continuous_model_path)
                
            loss_batch.clear()
            
        count += 1
        
    print("Epoch: ", epoch, ', l1 loss: ', np.average(loss_list))
    
    loss_list.clear()
    
    torch.save(model.state_dict(), continuous_model_path)
    
    save_checkpoint({
        'epoch': epoch + 1,
        'continuous_state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    })
    
print("Training finished")

Starting Training Loop...
Epoch:  0
Batch:  10 , l1 Loss:  0.6222708333622325 , Time: 14.060269832611084 s
Batch:  20 , l1 Loss:  0.33097506910562513 , Time: 26.45430064201355 s
Batch:  30 , l1 Loss:  0.28993759155273435 , Time: 38.93435883522034 s
Batch:  40 , l1 Loss:  0.23616625517606735 , Time: 51.451082706451416 s
Batch:  50 , l1 Loss:  0.22275505512952803 , Time: 64.02029180526733 s
Batch:  60 , l1 Loss:  0.20075249671936035 , Time: 76.67286324501038 s
Batch:  70 , l1 Loss:  0.18069818019866943 , Time: 89.28528094291687 s
Batch:  80 , l1 Loss:  0.18161795437335967 , Time: 101.93931722640991 s
Batch:  90 , l1 Loss:  0.17924284785985947 , Time: 114.61278223991394 s
Batch:  100 , l1 Loss:  0.14637547731399536 , Time: 127.29287838935852 s
Batch:  110 , l1 Loss:  0.1511179819703102 , Time: 140.0021674633026 s
Batch:  120 , l1 Loss:  0.1369858279824257 , Time: 152.76185870170593 s
Batch:  130 , l1 Loss:  0.14808819741010665 , Time: 165.49697756767273 s
Batch:  140 , l1 Loss:  0.1552505

Batch:  1140 , l1 Loss:  0.11564047709107399 , Time: 1467.037314414978 s
Batch:  1150 , l1 Loss:  0.11183306947350502 , Time: 1480.110852241516 s
Batch:  1160 , l1 Loss:  0.10603501051664352 , Time: 1493.1291942596436 s
Batch:  1170 , l1 Loss:  0.10788829177618027 , Time: 1506.0277047157288 s
Batch:  1180 , l1 Loss:  0.10977548584342003 , Time: 1518.9243104457855 s
Batch:  1190 , l1 Loss:  0.11410747244954109 , Time: 1531.8259069919586 s
Batch:  1200 , l1 Loss:  0.11226021870970726 , Time: 1544.7649977207184 s
Batch:  1210 , l1 Loss:  0.11147228777408599 , Time: 1557.6638133525848 s
Batch:  1220 , l1 Loss:  0.11225007697939873 , Time: 1570.6371417045593 s
Batch:  1230 , l1 Loss:  0.11264226734638214 , Time: 1583.596331357956 s
Batch:  1240 , l1 Loss:  0.11193570867180824 , Time: 1596.6141147613525 s
Batch:  1250 , l1 Loss:  0.11199744269251824 , Time: 1609.555477142334 s
Batch:  1260 , l1 Loss:  0.11478796526789665 , Time: 1622.5303087234497 s
Batch:  1270 , l1 Loss:  0.112783951312303

Batch:  2260 , l1 Loss:  0.10759318098425866 , Time: 2929.718319416046 s
Batch:  2270 , l1 Loss:  0.10892292857170105 , Time: 2942.836816072464 s
Batch:  2280 , l1 Loss:  0.10942751243710518 , Time: 2955.9783675670624 s
Batch:  2290 , l1 Loss:  0.10583948418498039 , Time: 2969.0882625579834 s
Batch:  2300 , l1 Loss:  0.10686551779508591 , Time: 2982.184993505478 s
Batch:  2310 , l1 Loss:  0.11189115643501282 , Time: 2995.226074695587 s
Batch:  2320 , l1 Loss:  0.10833612233400344 , Time: 3008.2703187465668 s
Batch:  2330 , l1 Loss:  0.11462832391262054 , Time: 3021.271742105484 s
Batch:  2340 , l1 Loss:  0.11157186627388001 , Time: 3034.3496351242065 s
Batch:  2350 , l1 Loss:  0.11146168783307076 , Time: 3047.395822286606 s
Batch:  2360 , l1 Loss:  0.10777470543980598 , Time: 3060.4134299755096 s
Batch:  2370 , l1 Loss:  0.10738113522529602 , Time: 3073.4846918582916 s
Batch:  2380 , l1 Loss:  0.10889825224876404 , Time: 3086.5090782642365 s
Batch:  2390 , l1 Loss:  0.10996963456273079

Batch:  3380 , l1 Loss:  0.10498561933636666 , Time: 4393.810075998306 s
Batch:  3390 , l1 Loss:  0.10814249664545059 , Time: 4406.872135877609 s
Batch:  3400 , l1 Loss:  0.10975121781229973 , Time: 4419.924778461456 s
Batch:  3410 , l1 Loss:  0.10419900193810464 , Time: 4433.002830982208 s
Batch:  3420 , l1 Loss:  0.10834634006023407 , Time: 4446.078515052795 s
Batch:  3430 , l1 Loss:  0.10692025274038315 , Time: 4459.1771504879 s
Batch:  3440 , l1 Loss:  0.10774853900074959 , Time: 4472.474814891815 s
Batch:  3450 , l1 Loss:  0.10331388339400291 , Time: 4485.553847551346 s
Batch:  3460 , l1 Loss:  0.10835448205471039 , Time: 4498.632539510727 s
Batch:  3470 , l1 Loss:  0.10801061168313027 , Time: 4511.690908670425 s
Batch:  3480 , l1 Loss:  0.10764374509453774 , Time: 4524.768872976303 s
Batch:  3490 , l1 Loss:  0.10752873122692108 , Time: 4537.801419496536 s
Batch:  3500 , l1 Loss:  0.10892132669687271 , Time: 4550.838408708572 s
Batch:  3510 , l1 Loss:  0.10953421369194985 , Time: 

Batch:  4510 , l1 Loss:  0.1072274774312973 , Time: 5876.233028411865 s
Batch:  4520 , l1 Loss:  0.10630022436380386 , Time: 5889.348572254181 s
Batch:  4530 , l1 Loss:  0.10596808195114135 , Time: 5902.486583948135 s
Batch:  4540 , l1 Loss:  0.10552069991827011 , Time: 5915.667801856995 s
Batch:  4550 , l1 Loss:  0.10414698347449303 , Time: 5928.802874326706 s
Batch:  4560 , l1 Loss:  0.10569369941949844 , Time: 5941.8566081523895 s
Batch:  4570 , l1 Loss:  0.1065135046839714 , Time: 5954.951425075531 s
Batch:  4580 , l1 Loss:  0.1053469367325306 , Time: 5968.05441403389 s
Batch:  4590 , l1 Loss:  0.10605354905128479 , Time: 5981.195420026779 s
Batch:  4600 , l1 Loss:  0.10657467693090439 , Time: 5994.337844848633 s
Batch:  4610 , l1 Loss:  0.1058957777917385 , Time: 6007.475194692612 s
Batch:  4620 , l1 Loss:  0.10367530211806297 , Time: 6020.61604642868 s
Batch:  4630 , l1 Loss:  0.10403775870800018 , Time: 6033.755775690079 s
Batch:  4640 , l1 Loss:  0.1033033013343811 , Time: 6046

...

Batch:  2550 , l1 Loss:  0.10131178423762321 , Time: 30826.39546251297 s
Batch:  2560 , l1 Loss:  0.10187182649970054 , Time: 30839.334251880646 s
Batch:  2570 , l1 Loss:  0.10205570757389068 , Time: 30852.265276432037 s
Batch:  2580 , l1 Loss:  0.09901227653026581 , Time: 30865.18859219551 s
Batch:  2590 , l1 Loss:  0.10131625384092331 , Time: 30878.106032848358 s
Batch:  2600 , l1 Loss:  0.10074106007814407 , Time: 30891.063526153564 s
Batch:  2610 , l1 Loss:  0.10112023651599884 , Time: 30904.0222697258 s
Batch:  2620 , l1 Loss:  0.09982427433133126 , Time: 30916.942880392075 s
Batch:  2630 , l1 Loss:  0.0986778162419796 , Time: 30929.774524450302 s
Batch:  2640 , l1 Loss:  0.10179335176944733 , Time: 30942.702518701553 s
Batch:  2650 , l1 Loss:  0.10148065909743309 , Time: 30955.597086429596 s
Batch:  2660 , l1 Loss:  0.10148991867899895 , Time: 30968.49508213997 s
Batch:  2670 , l1 Loss:  0.09933649301528931 , Time: 30981.435520887375 s
Batch:  2680 , l1 Loss:  0.10082468539476394

Batch:  3670 , l1 Loss:  0.09983877763152123 , Time: 32278.2036986351 s
Batch:  3680 , l1 Loss:  0.10030801892280579 , Time: 32291.236379623413 s
Batch:  3690 , l1 Loss:  0.10166999697685242 , Time: 32304.212918758392 s
Batch:  3700 , l1 Loss:  0.10088608115911483 , Time: 32317.250661849976 s
Batch:  3710 , l1 Loss:  0.10052293315529823 , Time: 32330.25079727173 s
Batch:  3720 , l1 Loss:  0.10248799994587898 , Time: 32343.15325665474 s
Batch:  3730 , l1 Loss:  0.09996562823653221 , Time: 32355.93921971321 s
Batch:  3740 , l1 Loss:  0.09995659813284874 , Time: 32368.75483727455 s
Batch:  3750 , l1 Loss:  0.09980555400252342 , Time: 32381.69346833229 s
Batch:  3760 , l1 Loss:  0.10138349756598472 , Time: 32394.670224428177 s
Batch:  3770 , l1 Loss:  0.10158933848142623 , Time: 32407.586256742477 s
Batch:  3780 , l1 Loss:  0.10122073739767075 , Time: 32420.503121614456 s
Batch:  3790 , l1 Loss:  0.10033605322241783 , Time: 32433.43492078781 s
Batch:  3800 , l1 Loss:  0.10133186429738998 ,

Batch:  4790 , l1 Loss:  0.09880012348294258 , Time: 33732.035818099976 s
Batch:  4800 , l1 Loss:  0.1016931876540184 , Time: 33745.055603027344 s
Batch:  4810 , l1 Loss:  0.10036587417125702 , Time: 33758.09735226631 s
Batch:  4820 , l1 Loss:  0.09778079539537429 , Time: 33771.05070948601 s
Batch:  4830 , l1 Loss:  0.10132867097854614 , Time: 33784.05187869072 s
Batch:  4840 , l1 Loss:  0.10134657323360444 , Time: 33797.10373330116 s
Batch:  4850 , l1 Loss:  0.09909924194216728 , Time: 33810.1434738636 s
Batch:  4860 , l1 Loss:  0.10084285438060761 , Time: 33823.17971134186 s
Batch:  4870 , l1 Loss:  0.10101465433835984 , Time: 33836.235557317734 s
Batch:  4880 , l1 Loss:  0.10023643150925636 , Time: 33849.25014424324 s
Batch:  4890 , l1 Loss:  0.1002196952700615 , Time: 33862.20684862137 s
Batch:  4900 , l1 Loss:  0.10008579641580581 , Time: 33875.242653131485 s
Batch:  4910 , l1 Loss:  0.10067836493253708 , Time: 33888.221528053284 s
Batch:  4920 , l1 Loss:  0.09923197478055953 , Ti

Batch:  5920 , l1 Loss:  0.10152223780751228 , Time: 35199.4715590477 s
Batch:  5930 , l1 Loss:  0.1007527120411396 , Time: 35212.367154598236 s
Batch:  5940 , l1 Loss:  0.10187013521790504 , Time: 35225.28562760353 s
Batch:  5950 , l1 Loss:  0.09913881719112397 , Time: 35238.324226379395 s
Batch:  5960 , l1 Loss:  0.09940312504768371 , Time: 35251.38167977333 s
Batch:  5970 , l1 Loss:  0.09797674715518952 , Time: 35264.38931107521 s
Batch:  5980 , l1 Loss:  0.10130699798464775 , Time: 35277.43057227135 s
Batch:  5990 , l1 Loss:  0.09882613718509674 , Time: 35290.45270204544 s
Batch:  6000 , l1 Loss:  0.10026866495609284 , Time: 35303.491309165955 s
Batch:  6010 , l1 Loss:  0.10060828775167466 , Time: 35316.60376572609 s
Batch:  6020 , l1 Loss:  0.09808481708168984 , Time: 35329.639719963074 s
Batch:  6030 , l1 Loss:  0.10112260282039642 , Time: 35342.67514538765 s
Batch:  6040 , l1 Loss:  0.09871230721473694 , Time: 35355.67322182655 s
Batch:  6050 , l1 Loss:  0.09989573955535888 , Ti

KeyboardInterrupt: 